<a href="https://colab.research.google.com/github/aubricot/computer_vision_with_eol_images/blob/master/object_detection_for_image_tagging/human_present/human_present_generate_tags_yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using YOLO v8 pre-trained on Google Open Images to add human present tags to EOL images
---
*Last Updated 12 September 2025*  
-Updated Sep 2025: Now runs in Python 3 with YOLOv8 pretrained on Google Open Images v7-   

Using a [YOLOv8 Nano](https://yolov8.com/) trained on [Open Images V7](https://docs.ultralytics.com/datasets/detect/open-images-v7) as a method to do customized, large-scale image processing. EOL Chiroptera (bat) images will be tagged for human(s) present (body, eye, head, hand, foot, face, arm, leg ear, eye, face, nose, beard) using object detection. Tags will further extend EOLv3 image search functions.

Notes:   
* Run code blocks by pressing play button in brackets on left
* Change parameters using form fields on right (find details at corresponding lines of code by searching '#@param')

References:   
* Code modified from the [Ultralytics YOLOv8 tutorial](https://docs.ultralytics.com/datasets/detect/open-images-v7)
* Check out the [Ultralytics YOLO repo](https://github.com/ultralytics/ultralytics) for more documetation and tutorials

## Installs & Imports
---

In [ ]:
#@title Choose where to save results
import os

# Use dropdown menu on right
save = "in Colab runtime (files deleted after each session)" #@param ["in my Google Drive", "in Colab runtime (files deleted after each session)"]

# TO DO: Define filter classes
filter_classes = ['Human', 'Person'] #@param

# Mount google drive to export image tagging file(s)
if 'Google Drive' in save:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

# Download helper_funcs folder
!pip3 -q install --upgrade gdown
!gdown 1yCB9SH5vJmleW84A3k7qVlkwpupp2F1q
!tar -xzvf helper_funcs_yolov8.tar.gz -C .

# Set up directory structure
from setup import *
cwd, basewd = setup_dirs() # Optional: specify basewd and folder to build your own pipelines

In [ ]:
#@title Import libraries
!uv pip install ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO
import os

# Load the pretrained model (YOLOv8 Nano trained on Open Images V7)
model = YOLO("yolov8n-oiv7.pt")

# For importing/exporting files, working with arrays, etc
import glob
import pathlib
import sys
import tarfile
import zipfile
import numpy as np
import csv
import time
import pandas as pd
import urllib.request

# For drawing onto and plotting images
import matplotlib.pyplot as plt
import cv2
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
sys.path.append('/content')

# Define EOL CV custom functions
from wrangle_data import *

## Generate tags for images
---
Run EOL 20k image bundles through pre-trained object detection models and save results in 4 batches (A-D) of 5000 images each.

In [ ]:
#@title Test: Run with sample EOL Chiroptera image (To test with your own image, upload file to data/imgs and update fn formfield)

# Run with sample EOL image
# Define image filepath and name
fn = "18.032f7703433402aa15bdccae63f5e94c.260x190.jpg" #@param ["18.032f7703433402aa15bdccae63f5e94c.260x190.jpg"] {allow-input: true}
img_fpath = 'data/imgs/' + fn

# Download image
%cd $cwd
%cd data/imgs
!gdown 10Ov02YgnjJo0gSQ0MC7B2o0WiJPl8rvN
%cd $cwd

# Run darknet and show bounding box coordinates
results = model(img_fpath)

# Display detection results
print("\nObject detection results:")
result_img = results[0].plot()  # returns an image (numpy array) with bounding boxes
plt.figure(figsize=(10, 8))
plt.imshow(cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("YOLOv8 Detection Results")
plt.show()

### Generate tags: Run inference on EOL images & save results for tagging - Run 4X for batches A-D
Use 20K EOL Chiroptera image bundle to identify humans present. Results are saved to [tags_file].tsv. Run this section 4 times (to make batches A-D) of 5K images each to incrementally save in case of Colab timeouts.

In [ ]:
#@title Enter EOL image bundle and choose inference settings. Change **tags_file** for each batch A-D
%cd $cwd

# Load in EOL Chiroptera 20k image bundle
bundle = "https://editors.eol.org/other_files/bundle_images/files/images_for_Chiroptera_breakdown_download_000001.txt" #@param ["https://editors.eol.org/other_files/bundle_images/files/images_for_Chiroptera_breakdown_download_000001.txt"] {allow-input: true}
df = read_datafile(bundle)
print("EOL image bundle with {} images: \n{}".format(len(df), df.head()))

# Test pipeline with a smaller subset than 5k images?
run = "test with tiny subset" #@param ["test with tiny subset", "for all images"]

# Display detection results on images?
display_results = "yes (use this option if testing tiny subsets; only works for \u003C50 images)" #@param ["yes (use this option if testing tiny subsets; only works for \u003C50 images)", "no (use this option if running batches)"]

# Take 5k subset of bundle for running inference
# Change filename for each batch
tags_file = "human_present_tags_c" #@param ["human_present_tags_a", "human_present_tags_b", "human_present_tags_c", "human_present_tags_d"] {allow-input: true}
tags_file = tags_file + ".txt"
imgs_dir = "data/img_info/"
outfpath = imgs_dir + tags_file
print("\nSaving tagging results to: \n{}".format(outfpath))

# Add 5k subset of image bundle urls as column in tags file
start, stop, cutoff = set_start_stop(run, df)
df = df.iloc[start:stop]
df.columns = ['url']
df.to_csv(outfpath, sep='\t', index=False)

In [ ]:
#@title Run inference on images

# Display detection results on image? (only use for <50 images)
# TO DO: Use form field on right to select/deselect
display_results = True #@param {type:"boolean"}

# Save image with bounding boxes?
# TO DO: Use form field on right to select/deselect
save_image_wboxes = True #@param {type:"boolean"}

print(f"\nNumber of valid images ready for inference: {len(df)}")
all_predictions = []
for i, (idx, row) in enumerate(df.iterrows()):
    try:
        image_url = row['url']
        print("Running infernce for image from: ", image_url)
        img_wboxes, tag, predictions = detect_and_draw_tags(image_url, tags = filter_classes)
        all_predictions.append(predictions)

        # Plot detections on image
        if display_results:
            plt.figure(figsize=(10, 10))
            plt.imshow(img_wboxes)
            plt.axis('off')
            plt.title(f"Detected {tag}")
            plt.show()

        # Save image with boxes
        if save_image_wboxes:
            # Convert back to BGR for saving with OpenCV
            img_bgr = cv2.cvtColor(img_wboxes, cv2.COLOR_RGB2BGR)
            # Define filename
            fname = os.path.splitext(image_url.split("/")[-1].split("?")[0])[0]  # Get name from URL
            fname = fname + "_w_bboxes.jpg"
            # Save image
            img_outfpath = os.path.join('/data/results', fname)
            cv2.imwrite(img_outfpath, img_bgr)
            print(f"Image with bounding boxes saved to to: {img_outfpath}")

    except Exception as e:
        print(f"Error processing {image_url}: {e}")

print("\n\n~~~\n \033[92m Inference complete! Post-process inference results in next code blocks before running these steps for remaining batches A-D. \033[0m \n~~~")

# Save rseults/bounding boxes to file
df_preds = pd.DataFrame(all_predictions, columns=["url", "tag", "xmin", "ymin", "bbox_w", "bbox_h", "confidence"])
df_preds.to_csv(('data/results/' + os.path.splitext(os.path.basename(outfpath))[0] + '.tsv'), sep='\t', index=False)
print(f"Predictions saved to: {('data/results/' + os.path.splitext(os.path.basename(outfpath))[0] + '.tsv')}")

## Combine tags from batches A-D
---
After running steps above for each image batch, combine tag files to one 20k tag dataset.

In [ ]:
#@title Define parameters for converting detected classes to desired image tags
%cd $cwd

# Write header row of output tagging file
# Enter any filename from 4 batches of tagging files
tags_file = "human_present_tags_d" #@param ["human_present_tags_d"] {allow-input: true}
tags_fpath = "data/results/" + tags_file + ".tsv"

# Combine exported model predictions and confidence values for all batches
df = combine_tag_files(tags_fpath)
df['tag'] = df['tag'].astype(str)

# Filter for desired classes
# These classes will be kept as is
filter1 = filter_classes
pattern1 = '|'.join(filter1)

# Inspect tags matching desired classes
print(f"\nNo. tags matching filtered class(es) {filter1}: {len(df[df.tag.str.match(pattern1)])}")
print("\nTags matching filtered class(es):")
print(df[df.tag.str.match(pattern1)].head())

df.loc[df['tag'].str.match(pattern1), 'tag'] = 'Human_present'

# Tag all unmatched predictions as "None"
print(f"\nNo. tags not matching filtered classes: {len(df[~df.tag.str.match(pattern1)])}")
print("\nTags not matching filtered classes:")
print(df[~df.tag.str.match(pattern1)].head())

df.loc[~df['tag'].str.match(pattern1), 'tag'] = 'None'

# Write results to tsv
outpath = 'data/results/' + tags_file.rsplit('_', 1)[0] + '_final.tsv'
df.to_csv(outpath, sep='\t', index=False)
print("\n\nFinal tagging file {}: \n{}".format(outpath, df.head()))

## Display tagging results on images
---

In [ ]:
#@title Adjust start index and display up to 50 images with their tags
# Suppress warning about too many figures opened
plt.rcParams.update({'figure.max_open_warning': 0})

# Set number of seconds to timeout if image url taking too long to open
import socket
socket.setdefaulttimeout(10)

# Adjust start index using slider
start = 0 #@param {type:"slider", min:0, max:5000, step:50}
stop = min((start+50), len(df))

# Loop through images
for i, row in df.iloc[start:stop].iterrows():
    try:
        # Read in image
        url = df['url'][i]
        img = load_image_from_url(url)

        # Fetch image tag
        tag = df['tag'][i]

        # Display progress message after each image is loaded
        print('Successfully loaded {} of {} images'.format(i+1, (stop-start)))

        # Plot image with tag
        _, ax = plt.subplots(figsize=(10, 10))
        ax.imshow(img)
        plt.axis('off')
        plt.title('{} ) Tag: {}'.format(i + 1, tag))

    except:
        print('Check if URL from {} is valid'.format(url))